In [7]:
import numpy as np
import glob
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import mplhep
import matplotlib.colors as mcolors
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
import pandas as pd

mplhep.style.use(mplhep.style.CMS)

In [178]:
def jsonload(fname):
    with open(fname) as jsonfile:
        try:
            return json.load(jsonfile)
        except Exception:
            print(fname)

In [179]:
fdir = "/Users/alexcampbell/Documents/Research/Fermilab/TID/Chip9_data"
fnames = list(np.sort(glob.glob(f"{fdir}/report*.json")))
all_files = fnames.copy()

In [180]:
to_remove = [
    f"{fdir}/report_2023-08-23T08:50:47.322524.json",
    f"{fdir}/report_2023-08-23T09:06:56.860898.json",
    f"{fdir}/report_2023-08-23T09:23:07.950039.json",
    f"{fdir}/report_2023-08-23T09:39:33.918227.json",
    f"{fdir}/report_2023-08-23T09:56:56.482843.json",
    f"{fdir}/report_2023-08-23T10:13:57.320867.json",
]
for i in range(len(all_files)):
    if all_files[i] in to_remove:
        fnames.remove(fnames[i])

In [181]:
data = [jsonload(fname) for fname in fnames]

for i in range(len(data)):
    if data[i] == None:
        data.remove(data[i])

In [182]:
data_nocap = data[:103]
data_cap = data[103:]

In [183]:
goodFnames = []
for j in range(len(data_nocap)):
    for i in range (len(data_nocap[j]['tests'])):
        if "metadata" in data_nocap[j]['tests'][i]:
            goodFnames.append(fnames[i])
            
Timestamps_nocap = []

for i in range(len(goodFnames)):
    tprim = goodFnames[i].split("_")[-1]
    timeb = tprim.split(".json")[0]
    times = timeb.split("T")
    timegood = times[0]+" "+times[1]
    Timestamps_nocap.append(timegood)
    

            

In [184]:
goodFnames = []
for j in range(len(data_cap)):
    for i in range (len(data_cap[j]['tests'])):
        if "metadata" in data_cap[j]['tests'][i]:
            goodFnames.append(fnames[i])
            
Timestamps_cap = []

for i in range(len(goodFnames)):
    tprim = goodFnames[i].split("_")[-1]
    timeb = tprim.split(".json")[0]
    times = timeb.split("T")
    timegood = times[0]+" "+times[1]
    Timestamps_cap.append(timegood)

# Before Decoupling Capacitor

In [185]:
voltages = []
frequencies = [42,43,44]
for j in range(len(data_nocap)):
    for i in range (len(data_nocap[j]['tests'])):
        if "metadata" in data_nocap[j]['tests'][i]:
            voltages.append(data_nocap[j]['tests'][i]['metadata']['voltage'])
            
spare1_asic = dict.fromkeys(list(np.unique(np.array(frequencies))))
spare1_emu = dict.fromkeys(list(np.unique(np.array(frequencies))))
for key in spare1_asic.keys():
    spare1_asic[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
    spare1_emu[key] = dict.fromkeys(list(np.unique(np.array(voltages))))


volts = list(np.unique(np.array(voltages)))

for volt in volts:
    for freq in frequencies:
        spare1_asic_list = []
        spare1_emu_list = []
        for j in range(len(data_nocap)):
            for i in range (len(data_nocap[j]['tests'])):
                if "metadata" in data_nocap[j]['tests'][i]:
                    if f'test_TID.py::test_streamCompareLoop[{volt}-{freq}]' in data_nocap[j]['tests'][i]['nodeid']:
                        spare1_emu_list.append(data_nocap[j]['tests'][i]['metadata']['spare1_emu'])
                        spare1_asic_list.append(data_nocap[j]['tests'][i]['metadata']['spare1_asic'])
                                          
        for i in range(len(spare1_asic_list)):
            for j in range(len(spare1_asic_list[0])):
                spare1_emu_list[i][j] = [f"{int(x):08x}" for x in spare1_emu_list[i][j]]
                spare1_asic_list[i][j] = [f"{int(x):08x}" for x in spare1_asic_list[i][j]]
                        
        spare1_asic[freq][volt] = spare1_asic_list
        spare1_emu[freq][volt] = spare1_emu_list

In [186]:
for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_asic[freq][volt])):
            spare1_asic[freq][volt][j] = np.array(spare1_asic[freq][volt][j])

In [187]:
for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_emu[freq][volt])):
            spare1_emu[freq][volt][j] = np.array(spare1_emu[freq][volt][j])

In [188]:
for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_asic[freq][volt])):
            spare1_asic[freq][volt][j] = spare1_asic[freq][volt][j][:,::-1]

In [189]:
for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_emu[freq][volt])):
            spare1_emu[freq][volt][j] = spare1_emu[freq][volt][j][:,::-1]

In [190]:
spare1_diffs = dict.fromkeys(list(np.unique(np.array(frequencies))))
for key in spare1_diffs.keys():
    spare1_diffs[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
for freq in frequencies:
    for volt in volts:
        spare1_diffs[freq][volt] = dict.fromkeys(list({
                                        'Location': None, 'Differences': None }))
    

In [191]:
for volt in volts:
    for freq in frequencies:
        locations = []
        differences = []
        for j in range(len(spare1_asic[freq][volt])):
            if np.argwhere(spare1_asic[freq][volt][j] != spare1_emu[freq][volt][j]).size != 0:
                locations.append(j)
                differences.append(np.argwhere(spare1_asic[freq][volt][j] != spare1_emu[freq][volt][j]))
        spare1_diffs[freq][volt]['Location'] = locations
        spare1_diffs[freq][volt]['Differences'] = differences
                

In [203]:
#for i in range(200):
#    print(f'{i:04d}   {"/".join(spare1_emu[42][1.08][85][i])}   {"/".join(spare1_asic[42][1.08][85][i])}') 

In [194]:
spare1_results = dict.fromkeys(list(np.unique(np.array(frequencies))))
spare1_results_timestamps = dict.fromkeys(list(np.unique(np.array(frequencies))))
for key in spare1_results.keys():
    spare1_results[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
    spare1_results_timestamps[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
for freq in frequencies:
    for volt in volts:
        spare1_results[freq][volt] = dict.fromkeys(list({
                                        '1': None, '2': None, '3': None, '4':None, '5':None, '6':None,'7':None}))
for freq in frequencies:
    for volt in volts:
        spare1_results_timestamps[freq][volt] = dict.fromkeys(list({
                                        '1': None, '2': None, '3': None, '4':None, '5':None, '6':None,'7':None}))

In [195]:
for freq in frequencies:
    for volt in volts:
        locations = []
        case_5 = 0
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size == 2:
                case_5 +=1
                locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])

        spare1_results[freq][volt]['5'] = case_5
        spare1_results_timestamps[freq][volt]['5'] = locations

In [196]:
for freq in frequencies:
    for volt in volts:
        case_1 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            length= 0
            for j in range(len(spare1_diffs[freq][volt]['Differences'][i])-1):
                if (spare1_diffs[freq][volt]['Differences'][i][j][0] != spare1_diffs[freq][volt]['Differences'][i][j+1][0] ):
                    length+=1
            if length == 199:
                case_1 += 1
                locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results[freq][volt]['1'] = case_1
        spare1_results_timestamps[freq][volt]['1'] = locations

In [197]:
for freq in frequencies:
    for volt in volts:
        case_3 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if(spare1_diffs[freq][volt]['Differences'][i][0][0] == spare1_diffs[freq][volt]['Differences'][i][1][0]) and (160 > spare1_diffs[freq][volt]['Differences'][i][0][0] > 155):
                    if ( 180> spare1_diffs[freq][volt]['Differences'][i][2][0] > 170):
                        if (195> spare1_diffs[freq][volt]['Differences'][i][-1][0] > 190):
                            case_3 += 1
                            locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results[freq][volt]['3'] = case_3
        spare1_results_timestamps[freq][volt]['3'] = locations

In [198]:
for freq in frequencies:
    for volt in volts:
        case_6 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if ( 140 > spare1_diffs[freq][volt]['Differences'][i][0][0] > 135):
                    if ( 149 >= spare1_diffs[freq][volt]['Differences'][i][-1][0] >= 145):
                        case_6 += 1
                        locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results[freq][volt]['6'] = case_6
        spare1_results_timestamps[freq][volt]['6'] = locations

In [199]:
for freq in frequencies:
    for volt in volts:
        case_4 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if ( 150 > spare1_diffs[freq][volt]['Differences'][i][0][0] > 120):
                    if (spare1_diffs[freq][volt]['Differences'][i][-1][0] == 199 ):
                        case_4 += 1
                        locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results[freq][volt]['4'] = case_4
        spare1_results_timestamps[freq][volt]['4'] = locations

In [200]:
for freq in frequencies:
    for volt in volts:
        case_7 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if (spare1_diffs[freq][volt]['Differences'][i][0][0] == 132):
                    if (spare1_diffs[freq][volt]['Differences'][i][-1][0] == 199 ):
                        case_7 +=1 
                        locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results[freq][volt]['7'] = case_7
        spare1_results_timestamps[freq][volt]['7'] = locations
                        
                    

In [201]:
length = 0
current_length = 0
lengths = []
for freq in frequencies:
    for volt in volts:
        case_2 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            for j in range(len(spare1_diffs[freq][volt]['Differences'][i])-1):
                if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                    if spare1_diffs[freq][volt]['Differences'][i][j][0] == spare1_diffs[freq][volt]['Differences'][i][j+1][0]:
                        length +=1
                        current_length = length
                    elif spare1_diffs[freq][volt]['Differences'][i][j][0] != spare1_diffs[freq][volt]['Differences'][i][j+1][0]:
                        length = 0 
                        l.append(current_length)
                        if current_length == 5 and (125 >= spare1_diffs[freq][volt]['Differences'][i][j-1][0] >= 120):
                            print(f"{freq} {volt} {i} {spare1_diffs[freq][volt]['Differences'][i][j-1][0]}")
                            case_2 +=1
                            locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
                        current_length = 0
                else:
                    l.append(0)
        spare1_results[freq][volt]['2'] = case_2
        spare1_results_timestamps[freq][volt]['2'] = locations
        lengths.append(l)


In [202]:
spare1_results


{42: {1.08: {'1': 3, '2': 0, '3': 0, '4': 50, '5': 1, '6': 2, '7': 3},
  1.11: {'1': 0, '2': 0, '3': 2, '4': 0, '5': 3, '6': 7, '7': 0},
  1.14: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0},
  1.2: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0},
  1.26: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 5, '6': 0, '7': 0},
  1.29: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 5, '6': 0, '7': 0},
  1.32: {'1': 0, '2': 0, '3': 1, '4': 0, '5': 0, '6': 0, '7': 0}},
 43: {1.08: {'1': 3, '2': 0, '3': 0, '4': 52, '5': 0, '6': 0, '7': 5},
  1.11: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 2, '6': 1, '7': 0},
  1.14: {'1': 0, '2': 0, '3': 1, '4': 0, '5': 0, '6': 0, '7': 0},
  1.2: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0},
  1.26: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 7, '6': 0, '7': 0},
  1.29: {'1': 0, '2': 0, '3': 1, '4': 0, '5': 2, '6': 0, '7': 0},
  1.32: {'1': 0, '2': 0, '3': 0, '4': 0, '5': 7, '6': 0, '7': 0}},
 44: {1.08: {'1': 1, '2': 0, '3': 0, '4': 54, '5': 0, '6': 1, '7':

# After Decoupling Capacitor

In [204]:
voltages = []
frequencies = [42,43,44]
for j in range(len(data_cap)):
    for i in range (len(data_cap[j]['tests'])):
        if "metadata" in data_cap[j]['tests'][i]:
            voltages.append(data_cap[j]['tests'][i]['metadata']['voltage'])
            
spare1_asic = dict.fromkeys(list(np.unique(np.array(frequencies))))
spare1_emu = dict.fromkeys(list(np.unique(np.array(frequencies))))
for key in spare1_asic.keys():
    spare1_asic[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
    spare1_emu[key] = dict.fromkeys(list(np.unique(np.array(voltages))))


volts = list(np.unique(np.array(voltages)))

for volt in volts:
    for freq in frequencies:
        spare1_asic_list = []
        spare1_emu_list = []
        for j in range(len(data_cap)):
            for i in range (len(data_cap[j]['tests'])):
                if "metadata" in data_cap[j]['tests'][i]:
                    if f'test_TID.py::test_streamCompareLoop[{volt}-{freq}]' in data_cap[j]['tests'][i]['nodeid']:
                        spare1_emu_list.append(data_cap[j]['tests'][i]['metadata']['spare1_emu'])
                        spare1_asic_list.append(data_cap[j]['tests'][i]['metadata']['spare1_asic'])
                                          
        for i in range(len(spare1_asic_list)):
            for j in range(len(spare1_asic_list[0])):
                spare1_emu_list[i][j] = [f"{int(x):08x}" for x in spare1_emu_list[i][j]]
                spare1_asic_list[i][j] = [f"{int(x):08x}" for x in spare1_asic_list[i][j]]
                        
        spare1_asic[freq][volt] = spare1_asic_list
        spare1_emu[freq][volt] = spare1_emu_list
for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_asic[freq][volt])):
            spare1_asic[freq][volt][j] = np.array(spare1_asic[freq][volt][j])
for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_emu[freq][volt])):
            spare1_emu[freq][volt][j] = np.array(spare1_emu[freq][volt][j])
for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_asic[freq][volt])):
            spare1_asic[freq][volt][j] = spare1_asic[freq][volt][j][:,::-1]

for volt in volts:
    for freq in frequencies:
        for j in range(len(spare1_emu[freq][volt])):
            spare1_emu[freq][volt][j] = spare1_emu[freq][volt][j][:,::-1]
spare1_diffs = dict.fromkeys(list(np.unique(np.array(frequencies))))
for key in spare1_diffs.keys():
    spare1_diffs[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
for freq in frequencies:
    for volt in volts:
        spare1_diffs[freq][volt] = dict.fromkeys(list({
                                        'Location': None, 'Differences': None }))
for volt in volts:
    for freq in frequencies:
        locations = []
        differences = []
        for j in range(len(spare1_asic[freq][volt])):
            if np.argwhere(spare1_asic[freq][volt][j] != spare1_emu[freq][volt][j]).size != 0:
                locations.append(j)
                differences.append(np.argwhere(spare1_asic[freq][volt][j] != spare1_emu[freq][volt][j]))
        spare1_diffs[freq][volt]['Location'] = locations
        spare1_diffs[freq][volt]['Differences'] = differences

spare1_results_cap = dict.fromkeys(list(np.unique(np.array(frequencies))))
spare1_results_timestamps_cap = dict.fromkeys(list(np.unique(np.array(frequencies))))
for key in spare1_results_cap.keys():
    spare1_results_cap[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
    spare1_results_timestamps_cap[key] = dict.fromkeys(list(np.unique(np.array(voltages))))
for freq in frequencies:
    for volt in volts:
        spare1_results_cap[freq][volt] = dict.fromkeys(list({
                                        '1': None, '2': None, '3': None, '4':None, '5':None, '6':None,'7':None}))
for freq in frequencies:
    for volt in volts:
        spare1_results_timestamps_cap[freq][volt] = dict.fromkeys(list({
                                        '1': None, '2': None, '3': None, '4':None, '5':None, '6':None,'7':None}))
for freq in frequencies:
    for volt in volts:
        locations = []
        case_5 = 0
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size == 2:
                case_5 +=1
                locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])

        spare1_results_cap[freq][volt]['5'] = case_5
        spare1_results_timestamps_cap[freq][volt]['5'] = locations
        
        
for freq in frequencies:
    for volt in volts:
        case_1 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            length= 0
            for j in range(len(spare1_diffs[freq][volt]['Differences'][i])-1):
                if (spare1_diffs[freq][volt]['Differences'][i][j][0] != spare1_diffs[freq][volt]['Differences'][i][j+1][0] ):
                    length+=1
            if length == 199:
                case_1 += 1
                locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results_cap[freq][volt]['1'] = case_1
        spare1_results_timestamps_cap[freq][volt]['1'] = locations
        
        
        
for freq in frequencies:
    for volt in volts:
        case_3 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if(spare1_diffs[freq][volt]['Differences'][i][0][0] == spare1_diffs[freq][volt]['Differences'][i][1][0]) and (160 > spare1_diffs[freq][volt]['Differences'][i][0][0] > 155):
                    if ( 180> spare1_diffs[freq][volt]['Differences'][i][2][0] > 170):
                        if (195> spare1_diffs[freq][volt]['Differences'][i][-1][0] > 190):
                            case_3 += 1
                            locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results_cap[freq][volt]['3'] = case_3
        spare1_results_timestamps_cap[freq][volt]['3'] = locations
        
for freq in frequencies:
    for volt in volts:
        case_6 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if ( 140 > spare1_diffs[freq][volt]['Differences'][i][0][0] > 135):
                    if ( 149 >= spare1_diffs[freq][volt]['Differences'][i][-1][0] >= 145):
                        case_6 += 1
                        locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results_cap[freq][volt]['6'] = case_6
        spare1_results_timestamps_cap[freq][volt]['6'] = locations
        
        
for freq in frequencies:
    for volt in volts:
        case_4 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if ( 150 > spare1_diffs[freq][volt]['Differences'][i][0][0] > 120):
                    if (spare1_diffs[freq][volt]['Differences'][i][-1][0] == 199 ):
                        case_4 += 1
                        locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results_cap[freq][volt]['4'] = case_4
        spare1_results_timestamps_cap[freq][volt]['4'] = locations

for freq in frequencies:
    for volt in volts:
        case_7 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                if (spare1_diffs[freq][volt]['Differences'][i][0][0] == 132):
                    if (spare1_diffs[freq][volt]['Differences'][i][-1][0] == 199 ):
                        case_7 +=1 
                        locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
        spare1_results_cap[freq][volt]['7'] = case_7
        spare1_results_timestamps_cap[freq][volt]['7'] = locations

length = 0
current_length = 0
lengths = []
for freq in frequencies:
    for volt in volts:
        case_2 = 0
        locations = []
        for i in range(len(spare1_diffs[freq][volt]['Differences'])):
            for j in range(len(spare1_diffs[freq][volt]['Differences'][i])-1):
                if spare1_diffs[freq][volt]['Differences'][i].size > 2:
                    if spare1_diffs[freq][volt]['Differences'][i][j][0] == spare1_diffs[freq][volt]['Differences'][i][j+1][0]:
                        length +=1
                        current_length = length
                    elif spare1_diffs[freq][volt]['Differences'][i][j][0] != spare1_diffs[freq][volt]['Differences'][i][j+1][0]:
                        length = 0 
                        l.append(current_length)
                        if current_length == 5 and (125 >= spare1_diffs[freq][volt]['Differences'][i][j-1][0] >= 120):
                            print(f"{freq} {volt} {i} {spare1_diffs[freq][volt]['Differences'][i][j-1][0]}")
                            case_2 +=1
                            locations.append(Timestamps_nocap[spare1_diffs[freq][volt]['Location'][i]])
                        current_length = 0
                else:
                    l.append(0)
        spare1_results_cap[freq][volt]['2'] = case_2
        spare1_results_timestamps_cap[freq][volt]['2'] = locations
        lengths.append(l)
       

# Creating Dataframe

In [218]:
nocap = pd.DataFrame.from_dict(spare1_results)
nocap_timestamps = pd.DataFrame.from_dict(spare1_results_timestamps)
frames = [nocap, nocap_timestamps]
no_capresult = pd.concat(frames)

In [219]:
cap = pd.DataFrame.from_dict(spare1_results_cap)
cap_timestamps = pd.DataFrame.from_dict(spare1_results_timestamps_cap)
frames = [cap, cap_timestamps]
capresult = pd.concat(frames)